<a href="https://colab.research.google.com/github/merucode/DL/blob/92-Colab-Kaggle-ML-Classification/01-02_%5BClassification-LR%5D_Categorical-feature-encoding(improvement).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imformation

* Title : [Categorical Feature Encoding](https://www.kaggle.com/c/cat-in-the-dat)
* Type : Binary Classification
* Evaluation : ROC AUC
* Model : Logistic Regresion
* Python version: 3.10.6
* Basic library version
  * sklearn(scikit-learn==1.2.2)
  * numpy(numpy==1.22.4)
  * pandas(pandas==1.5.3)
  * matplotlib(matplotlib==3.7.1)
  * seaborn(seaborn==0.12.2)
  * scipy(scipy==1.10.1)
* Addtional Library version
* Considering Library version
* Improvement
  * Feature Set Encoding
  * Feature Scaling
  * Grid Search

# STEP 0. Version check and Install Dependency

Step 0-1. Install Dependency

Step 0-2. Version Check

In [ ]:
import sys
import torch
print(f"Python version:{sys.version}")                  # python
print("Torch version:{}".format(torch.__version__))     # torch
print("cuda version: {}".format(torch.version.cuda))    # cuda
print("cudnn version:{}".format(torch.backends.cudnn.version()))    # cudnn

In [ ]:
!pip list

Step 0-3. Download Data

In [ ]:
!export KAGGLE_USERNAME=*** && export KAGGLE_KEY=*** && kaggle competitions download -c cat-in-the-dat

In [ ]:
from zipfile import ZipFile

data_path = '/content/'

with ZipFile(data_path + 'cat-in-the-dat.zip') as zipper:
  zipper.extractall()

# STEP 1. Check Data

★ 분석결과
* 이진 피쳐
  * bin_3: T → 1, F → 0 인코딩
  * bin_4: Y → 1, N → 0 인코딩
* 순서형 피쳐
  * 고윳값 순서에 맞게 인코딩

Step 1-1. Check data

In [ ]:
import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/content/'

# 훈련, 검증, 테스트 데이터 경로 설정
train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

In [ ]:
train.shape, test.shape, submission.shape

In [ ]:
train.head().T

In [ ]:
test.head().T

In [ ]:
submission.head()

In [ ]:
train.info()

In [ ]:
### 피쳐 요약표
def resumetable(df):
  print(f"데이터셋 형상: {df.shape}")
  summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
  summary = summary.reset_index()
  summary = summary.rename(columns={'index': '피쳐'})
  summary['결측값 갯수'] = df.isnull().sum().values
  summary['고유값 개수'] = df.nunique().values
  summary['첫 번째 값'] = df.loc[0].values
  summary['두 번째 값'] = df.loc[1].values
  summary['세 번째 값'] = df.loc[2].values

  return summary

resumetable(train)

In [ ]:
### 순서형 데이터 고윳값 확인
for i in range(3):
  feature = 'ord_' + str(i)
  print(f'{feature} 고윳값: {train[feature].unique()}')

for i in range(3, 6):
  feature = 'ord_' + str(i)
  print(f'{feature} 고윳값: {train[feature].unique()}')

In [ ]:
### 그 외 데이터 고윳값 확인
check_list = ['day', 'month', 'target']
for i in check_list:
  print(f'{i} 고윳값: {train[i].unique()}')

Step 1-2. Features Info

* 이진 피쳐 : bin_0 ~ bin_4
* 명목형 피쳐 : nom_0 ~ nom_9
* 순서형 피쳐 : ord_0 ~ ord_5
* 그 외 피쳐 : day, month, target

★ 분석결과
* 이진 피쳐
  * bin_3: T → 1, F → 0 인코딩
  * bin_4: Y → 1, N → 0 인코딩
* 순서형 피쳐
  * 고윳값 순서에 맞게 인코딩

Step 1-3. Feature Engineering

# STEP 2. Data Visualize
★ 분석결과
* day, month : 원-핫 인코딩 필요

STEP 2-1. Count Plot

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
### Count Plot : 범주형 데이터 갯수 확인
mpl.rc('font', size=15) # 폰트 크기를 15로 설정
plt.figure(figsize=(3, 3))

# 타깃값 분포 카운트플롯
ax = sns.countplot(x='target', data=train)
ax.set_title('Target Distributuion')

In [ ]:
### 네모 도형 객체를 순회화며 막대 상단에 타깃값 비율 표시
def write_percent(ax, total_size):
  for patch in ax.patches:
    height = patch.get_height() # 도형 높이(데이터 개수)
    width = patch.get_width()   # 도형 너비
    left_coord = patch.get_x()  # 도형 왼쪽 테두리의 x축 위치
    percent = height/total_size*100

    # x,y 좌표에 텍스트 입력
    ax.text(x=left_coord + width/2.0,
            y=height + total_size*0.001,
            s=f'{percent:1.1f}%',
            ha='center')

STEP 2-2. Bar Plot

In [ ]:
import matplotlib.gridspec as gridspec # 여러 그래프를 격자 형태로 배치
### BAR PLOT : 범주형 데이터에 따른 수치형 데이터 정보

# m행 n열 Figure 준비
mpl.rc('font', size=12)       # 폰트 크기 설정
grid = gridspec.GridSpec(3,2) # 그래프 3*2 배치
plt.figure(figsize=(10, 16))
plt.subplots_adjust(wspace=0.4, hspace=0.3) # 서브플롯 간 좌우/상하 여백 설정

# 서브플롯 그리기
bin_features = ['bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4']  # 피처 목록

for idx, feature in enumerate(bin_features):
  ax = plt.subplot(grid[idx])

  # ax축에 타깃값 분포 카운트플롯 그리기
  sns.countplot(x=feature,
                data=train,
                hue='target',
                palette='pastel',
                ax=ax)

  ax.set_title(f'{feature} Distribution by Target')
  write_percent(ax, len(train))

Step 2-3. 교차분석표

In [ ]:
### Cross-tabulation: 범주형 데이터 2개를 비교 분석
# 교차분석표 df 생성 함수 정의
def get_crosstab(df, feature):
  crosstab = pd.crosstab(df[feature], df['target'], normalize='index') * 100
  crosstab = crosstab.reset_index()
  return crosstab

In [ ]:
### Point Plot 함수 정의

def plot_pointplot(ax, feature, crosstab): # 카운트플롯이 그려진 축에 포인트플롯 중복으로 그리는 함수
  ax2 = ax.twinx()  # x축은 공유하고 y축은 공유하지 않는 새로운 축 생성
  # 새로운 축에 포인트플롯 그리기
  ax2 = sns.pointplot(x=feature, y=1, data=crosstab,
                      order=crosstab[feature].values, # 포인트플롯 순서
                      color='black')
  ax2.set_ylim(crosstab[1].min()-5, crosstab[1].max()*1.1)  # y축 범위 설정
  ax2.set_ylabel('Target 1 Ratio(%)')

In [ ]:
### 피처분포도 및 피처별 타깃값 1의 비율 포인트플롯 생성함수 만들기
def plot_cat_dist_with_true_ratio(df, features, num_rows, num_cols, size=(15,20)):
  plt.figure(figsize=size)  # Figure 크기
  grid = gridspec.GridSpec(num_rows, num_cols)  # 서브플롯 배치
  plt.subplots_adjust(wspace=0.45, hspace=0.3)  # 서브플롯 여백

  for idx, feature in enumerate(features):
    ax = plt.subplot(grid[idx])
    crosstab = get_crosstab(df, feature)  # 교차분석표 생성

    # ax축에 타깃값 분포 카운트플롯 그리기
    sns.countplot(x=feature, data=df,
                  order=crosstab[feature].values,
                  color='skyblue',
                  ax=ax)
    write_percent(ax, len(df))  # 비율 표시

    plot_pointplot(ax, feature, crosstab) # 포인트플롯 그리기

    ax.set_title(f'{feature} Distribution')

In [ ]:
nom_features = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']
plot_cat_dist_with_true_ratio(train, nom_features, num_rows=3, num_cols=2)

In [ ]:
ord_features = ['ord_0', 'ord_1', 'ord_2', 'ord_3'] # 순서대로 정렬되어 있지 않음

plot_cat_dist_with_true_ratio(train, ord_features, num_rows=2, num_cols=2, size=(10,10))

In [ ]:
### ord_feature 순서대로 정렬
from pandas.api.types import CategoricalDtype

ord_1_value = ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster']
ord_2_value = ['Freezing', 'Cold', 'Warm', 'Hot', 'Boiling Hot', 'Lava Hot']

# 순서를 지정한 범주형 데이터 타입
ord_1_dtype = CategoricalDtype(categories=ord_1_value, ordered=True)
ord_2_dtype = CategoricalDtype(categories=ord_2_value, ordered=True)

# 데이터 타입 변경
train['ord_1'] = train['ord_1'].astype(ord_1_dtype)
train['ord_2'] = train['ord_2'].astype(ord_2_dtype)

plot_cat_dist_with_true_ratio(train, ord_features, num_rows=2, num_cols=2, size=(10,10))

In [ ]:
plot_cat_dist_with_true_ratio(train, ['ord_4', 'ord_5'], num_rows=2, num_cols=1, size=(15,12))

In [ ]:
date_features = ['day', 'month']
plot_cat_dist_with_true_ratio(train, date_features, num_rows=2, num_cols=1, size=(10,10))

Step 2-7. ★ 분석결과

* day, month : 원-핫 인코딩 필요

# STEP 3. Feature Engineering
### ★ Feature Set Encoding

Step 3-1. Load Data

In [ ]:
import numpy as np
import pandas as pd

# 데이터 경로
data_path = '/content/'

# 훈련, 검증, 테스트 데이터 경로 설정
train = pd.read_csv(data_path + 'train.csv', index_col='id')
test = pd.read_csv(data_path + 'test.csv', index_col='id')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col='id')

Step 3-2. Concat Data(Apply same feature engineering with train, test)

In [ ]:
all_data = pd.concat([train, test]) # 훈련 데이터와 테스트 데이터 합치기
all_data = all_data.drop('target', axis=1)  # 타깃값 제거
all_data.tail(3)

Step 3-3. ★ Binary Set Encoding

In [ ]:
all_data['bin_3'] = all_data['bin_3'].map({'F':0, 'T':1})
all_data['bin_4'] = all_data['bin_4'].map({'N':0, 'Y':1})

Step 3-4. ★ Ord Set Encoding

In [ ]:
ord1dict = {'Novice':0, 'Contributor':1, 'Expert':2, 'Master':3, 'Grandmaster':4}
ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2, 'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}

all_data['ord_1'] = all_data['ord_1'].map(ord1dict)
all_data['ord_2'] = all_data['ord_2'].map(ord2dict)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_345 = ['ord_3', 'ord_4', 'ord_5']

ord_encoder = OrdinalEncoder()  # OrdinalEncoder 객체 생성
# ordinal 인코딩 적용
all_data[ord_345] = ord_encoder.fit_transform(all_data[ord_345])

# 피처별 인코딩 순서 출력
for feature, categories in zip(ord_345, ord_encoder.categories_):
  print(feature)
  print(categories)

Step 3-5. Nom Set Encoder(One-Hot Encoding)

In [ ]:
nom_features = ['nom_' + str(i) for i in range(10)]

from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()   # 원핫인코더 생성

# all_data[nom_features] = onehot_encoder.fit_transform(all_data[nom_features]) # 오류 발생(곧바로 인코딩X)
encoded_nom_matrix = onehot_encoder.fit_transform(all_data[nom_features])

encoded_nom_matrix

In [ ]:
# 기존 데이터의 기존 명목형 피쳐 삭제(Onehot 인코더에 모두 정보 들어있기 때문에)
all_data = all_data.drop(nom_features, axis=1)

Step 3-6. ETC Set Encoder(One-Hot Encoding)

In [ ]:
date_features = ['day', 'month']

encoded_date_matrix = onehot_encoder.fit_transform(all_data[date_features])

all_data = all_data.drop(date_features, axis=1)

encoded_date_matrix

Step 3-7. ★ Feature Scaling

In [ ]:
### 순서형 min-max 정규화
from sklearn.preprocessing import MinMaxScaler

ord_features = ['ord_' + str(i) for i in range(6)]
# min-max 정규화
all_data[ord_features] = MinMaxScaler().fit_transform(all_data[ord_features])

Step 3-8. ★ Merge Features

In [ ]:
from scipy import sparse

# 인코딩 및 스케일링 된 피처 합치기 > 모두 CSR 전환
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data),
                               encoded_nom_matrix,
                               encoded_date_matrix],
                              format='csr')

all_data_sprs

Step 3-4. Divide Data(train, test, valid)

In [ ]:
num_train = len(train)  # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target']

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련/검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1,    # 9:1
                                                      stratify=y,       # 타깃값 비율 일정하게
                                                      random_state=10)

# STEP 4. Model


In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression() # 모델 생성

# STEP 5. Learning
### ★ Grid Search

Step 5-1. Setting

Step 5-2. Learning + ★ Grid Search

In [ ]:
%%time

from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 값 목록
lr_params = {'C':[0.1, 0.125, 0.2], 'max_iter':[800, 900, 1000],
             'solver':['liblinear'], 'random_state':[42]}
# 그리드서치 객체 생성
gridsearch_logistic_model = GridSearchCV(estimator=logistic_model,
                                         param_grid=lr_params,
                                         scoring='roc_auc', # 평가지표
                                         cv=5)

# 그리드서치 수행
gridsearch_logistic_model.fit(X_train, y_train)  # 모델 훈련

print(f"최적 하이퍼파라미터: {gridsearch_logistic_model.best_params_}")

# STEP 6. Validation

In [ ]:
# logistic_model.predict_proba(X_valid) # 0 또는 1 예측 확률
# logistic_model.predict(X_valid)       # 0 또는 1 예측 결과

# 검증데이터를 통한 타깃값이 1일 확률 예측
y_valid_preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_valid)[:, 1]

In [ ]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC ACU
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f"선형 회귀 RMSLE 값: {roc_auc:.4f}")

# STEP 7. Prediction and Submission

Step 7-1. Prediction

In [ ]:
# 타깃값 1일 확률 예측
preds = gridsearch_logistic_model.best_estimator_.predict_proba(X_test)[:, 1]

Step 7-2. Submission

In [ ]:
submission['target'] = preds
submission.to_csv('submission.csv', index=False)   # 제출 파일 생성